<a href="https://colab.research.google.com/github/PedroFerreiradaCosta/FaceFitOpt/blob/master/FaceFitOpt_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown .
%%html
<marquee style='width: 30%; color: green;'><b>&lt;&lt;&lt; Let's start by pressing the Play button next to 'Face Fit Opt' inside the [ ] on the left &lt;&lt;&lt;</b></marquee>

In [ ]:
from IPython.utils import io
#@markdown # **Face Fit Opt**
import time
from IPython.display import display, Javascript, HTML
from ipywidgets import widgets, Layout
with io.capture_output() as captured:
  !pip install bayesian-optimization
  %tensorflow_version 1.x
  !git clone https://github.com/PedroFerreiradaCosta/stylegan-encoder.git
  %load_ext autoreload
  %autoreload 2
import os
os.chdir("stylegan-encoder")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import pickle
from ipywidgets import widgets
import PIL.Image
import numpy as np
import dnnlib
import dnnlib.tflib as tflib
import config
from encoder.generator_model import Generator
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output
import bayes_opt
import tensorflow as tf
import warnings
from sklearn.gaussian_process.kernels import Matern, WhiteKernel
from google.colab.output import eval_js
from base64 import b64decode
from bayes_opt import BayesianOptimization, UtilityFunction
from ipywidgets import interact, interactive, fixed, interact_manual
from vis import plot_bo
from itertools import product

# display(HTML("<marquee style='width: 30%; color: green;'><b>&lt;&lt;&lt; Let's start by pressing the Play button inside the [ ] on the left &lt;&lt;&lt;</b></marquee>"))

run = widgets.ToggleButton(
    value=False,
    description="Let's Go!",
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click when ready to start game.',
    layout= Layout(border = '1px solid black')
)


display(run)

def run_clicked(obj):
    display(HTML("<marquee style='width: 30%; color: green;'><b>&lt;&lt;&lt; Wait for camera &lt;&lt;&lt;</b></marquee>"))
    run.close()
    main()

run.observe(run_clicked, 'value')

def generate_image(latent_vector):
    latent_vector = latent_vector.reshape((1, 18, 512))
    generator.set_dlatents(latent_vector)
    img_array = generator.generate_images()[0]
    img = PIL.Image.fromarray(img_array, 'RGB')
    #return img
    return img.resize((256, 256))


def move_and_show(latent_vector, direction, coeffs):
    fig,ax = plt.subplots(1, len(coeffs), figsize=(15, 10), dpi=80)
    for i, coeff in enumerate(coeffs):
        new_latent_vector = latent_vector.copy()
        new_latent_vector[:8] = (latent_vector + coeff*direction)[:8]
        ax[i].imshow(generate_image(new_latent_vector))
        ax[i].set_title('Coeff: %0.1f' % coeff)
    [x.axis('off') for x in ax]
    plt.show()

def plot_sampled_face(latent_vector, direction1, direction2, x, y, i):

  clear_output(wait=True)
  #display(plt.gcf())
  plt.figure(figsize=(10,10))
  new_latent_vector = latent_vector.copy()
  new_latent_vector[:8] = (latent_vector + x*direction1)[:8]
  new_latent_vector[:8] = (new_latent_vector + y*direction2)[:8]
  plt.imshow(generate_image(new_latent_vector))
  plt.title('Iteration: {} of 26'.format(i+1))
  plt.axis('off')
  plt.show()

# Loading already learned latent directions
smile_direction = np.load('ffhq_dataset/latent_directions/smile.npy')
age_direction = np.load('ffhq_dataset/latent_directions/age.npy')


# Create Generator Instance
# path to model code and weight
path_model = 'https://drive.google.com/uc?id=1I1vyhKJhoQul3ryIrHcTlpYycPAhWooI'

#   """ create tf session """
yn_CPU_only = False
gpu_fraction = 0.5
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)

if yn_CPU_only:
    config = tf.ConfigProto(device_count = {'GPU': 0}, allow_soft_placement=True)
else:
    config = tf.ConfigProto(allow_soft_placement=True, gpu_options=gpu_options)
    config.gpu_options.allow_growth = True

sess = tf.InteractiveSession(config=config)

with io.capture_output() as captured:
  try:
      with dnnlib.util.open_url(path_model, 'rb') as file:
          G, D, Gs = pickle.load(file)
  except FileNotFoundError:
      print('before running the code, download pre-trained model to project_root/asset_model/')
      raise
  generator = Generator(Gs, batch_size=1, randomize_noise=False)


def main():
  clear_output()

  os.mkdir('raw_images')
  os.mkdir('latent_representations')
  os.mkdir('aligned_images')
  os.mkdir('generated_images')

  ###############################################################################


  def take_photo(filename='raw_images/photo.jpg', quality=0.8):
    clear_output()
    js = Javascript('''
      async function takePhoto(quality) {
        const div = document.createElement('div');
        const capture = document.createElement('button');

        capture.style.width = '200px'; // setting the width to 200px
        capture.style.height = '50px'; // setting the height to 50px
        capture.style.background = 'green'; // setting the background color to green
        capture.style.color = 'white'; // setting the color to white
        capture.style.fontSize = '20px'; // setting the font size to 20px
        capture.textContent = 'Capture';

        div.appendChild(capture);

        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();

        // Resize the output to fit the video element.
        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

        // Wait for Capture to be clicked.
        await new Promise((resolve) => capture.onclick = resolve);

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getVideoTracks()[0].stop();
        div.remove();
        return canvas.toDataURL('image/jpeg', quality);
      }
      ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
      f.write(binary)
    return filename

  print('')  
  print("Press Capture below to take a photo")
  print("Try to avoid accessories like glasses or hats.")

  from IPython.display import Image
  try:
    filename = take_photo()
    
  except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not
    # grant the page permission to access it.
    print(str(err))
    print('You must have a webcam and grant permission to the program to use it.')
    !kill -9 -1

  ###############################################################################
  with io.capture_output() as captured:
    # 1) Extract and align faces from images
    !rmdir raw_images/.ipynb_checkpoints/
    # Images to be processed should be added to the folder 'raw_images'
    !python align_images.py raw_images/ aligned_images/
    # if downloading dlib face landmarks takes more than 1/2 min, rerun code
  print('')
  print('')
  display(HTML("<marquee style='width: 30%; color: green;'><b>&lt;&lt;&lt; Hello again! Right now the face machine is finding an artificial photo that looks just like yours to play in the *Space Face*. It will do that using Artificial Intelligence. This will take between 5 and 10 minutes, so feel free to go make yourself a cup of tea........................................................................Still around? Good! What you will be asked to do next is rate some faces if they look like you or not. Some will be very similar. Others.... not so much. Remember, there are no wrong answers, just what you feel like its the right rating! The machine also makes mistakes, so dont worry if the faces dont look anything like you. </b></marquee>"))


  # 2) Find latent representation of aligned images
  with io.capture_output() as captured:
    !python encode_images.py aligned_images/ generated_images/ latent_representations/ --iterations 500

  # src_dir - directory with images
  # generated_images_dir - directory for storing generated images
  # dlatent_dir - directory for storing dlatent representations
  # batch_size - df - 1 - for generator and perceptual model
  # image_size - df - 256 - Size of images for perceptual model
  # lr - df 1 - learning rate for perc. model
  # iterations - df - 1000 - opetimizations steps per batch 
  # randomize_noise-  df - False - add noise to dlatents during optimization

  run_bayes(generator) #Runs bayesian optimization on the space created by the image
  ###############################################################################

def run_bayes(generator):

  # Latent on the origin of the Space (0,0)
  latent_vector = np.load('latent_representations/photo_01.npy')

  ###############################################################################

  generator = generator
  def f(rating):
    optimizer.register(next_point_to_probe, rating)
    
    return rating

  pbounds = {'x': (-1, 1), 'y': (-1, 1)}
  optimizer = BayesianOptimization(
      f=None,
      pbounds=pbounds,
      verbose=2,
      random_state=42,
  )
  kernel = 1.0 * Matern(length_scale=0.20, length_scale_bounds=(0.05, 1), nu=2.5)\
          + WhiteKernel(noise_level=50, noise_level_bounds=(1e-10, 0.5))
  optimizer.set_gp_params(kernel=kernel)
  
  utility = UtilityFunction(kind="ucb", kappa=2, xi=0)

  ###############################################################################

  init_points = 5
  n_iter = 21
  
  def sample_and_register(sample):
      print('sample and register: {}'.format(sample))
      if sample < init_points:
          next_point_to_probe = {'x': np.random.uniform(pbounds['x'][0], pbounds['x'][1]),
                                  'y': np.random.uniform(pbounds['y'][0], pbounds['y'][1])}

          x, y = next_point_to_probe.values()
      else:
          next_point_to_probe = optimizer.suggest(utility)
          x, y = next_point_to_probe.values()

      # Here I show the image generated by the new latent
      plot_sampled_face(latent_vector, smile_direction, age_direction, x*2, y*2, sample)

      rating_input = widgets.BoundedFloatText(
      value=5.0,
      min=0,
      max=10.0,
      step=0.1,
      disabled=False
      )
      print('How much does this image look like you? Rate it in the box below from 0 to 10, 10 being just like you.\n You can use decimals (e.g. 8.7).\n')
      display(rating_input)

      run = widgets.ToggleButton(
      value=False,
      description="Rate!",
      disabled=False,
      button_style='success', # 'success', 'info', 'warning', 'danger' or ''
      tooltip='Click when you have evaluated the face.',
      layout= Layout(border = '1px solid black')
      )

      display(run)

      def run_clicked(obj): # New sample, register and start again
        if sample + 1 < init_points + n_iter:
          
          rating = rating_input.value
          optimizer.register(next_point_to_probe, float(rating))
          sample_and_register(sample+1)
          run.close()
          rating_input.close()
        else:
          end_sample(optimizer, latent_vector)

      run.observe(run_clicked, 'value')

  sample_and_register(0)

def end_sample(optimizer, latent_vector):
  maximum, params = optimizer.max.values()
  x_max,y_max = params.values()

  ###############################################################################

  x = np.linspace(-1, 1, 50).reshape(-1, 1)
  y = np.linspace(-1, 1, 50).reshape(-1, 1)

  x_obs = np.array([[res["params"]["x"]] for res in optimizer.res])
  y_obs = np.array([[res["params"]["y"]] for res in optimizer.res])
  z_obs = np.array([res["target"] for res in optimizer.res])

  x1x2 = np.array(list(product(x, y)))
  X0p, X1p = x1x2[:, 0].reshape(50, 50), x1x2[:, 1].reshape(50, 50)

  X = (np.array([x_obs.ravel(), y_obs.ravel()])).T
  np.save('values',np.array([x_obs.ravel(), y_obs.ravel(), z_obs.ravel()]).T)
  
  #optimizer._gp.fit(X, z_obs)
  mu, std = optimizer._gp.predict(x1x2.reshape(-1, 2), return_std=True)
  np.save('mu',mu)
  np.save('std',std)

  ###############################################################################

  clear_output()

  run = widgets.ToggleButton(
  value=False,
  description="Results",
  disabled=False,
  button_style='info', # 'success', 'info', 'warning', 'danger' or ''
  tooltip='Click to see your results.',
  layout= Layout(border = '1px solid black')
  )

  display(HTML("<marquee style='width: 30%; color: green;'><b>You did it! Well done! Do you want to see what your results look like? Click on the button! Also, please consider leaving a star in the Github Repo. Thank you!</b></marquee>"))

  display(run)

  def run_clicked(obj): # Clicked to see results
      run.close()
      clear_output()
      plot_result(optimizer, latent_vector)


  run.observe(run_clicked, 'value')



def plot_result(optimizer, latent_vector):
  # #### PLOT RESULTS
  from sklearn.gaussian_process import GaussianProcessRegressor
  from sklearn.gaussian_process.kernels import Matern, WhiteKernel
  x = np.linspace(-1, 1, 50).reshape(-1, 1)
  y = np.linspace(-1, 1, 50).reshape(-1, 1)

  x_obs = np.array([[res["params"]["x"]] for res in optimizer.res])
  y_obs = np.array([[res["params"]["y"]] for res in optimizer.res])
  z_obs = np.array([res["target"] for res in optimizer.res])
  x1x2 = np.array(list(product(x, y)))

  kernel=2.09**2 * Matern(length_scale=1, nu=2.5) + WhiteKernel(noise_level=0.5)
  gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, optimizer=None)

  gpr.fit(np.array([x_obs,y_obs]).reshape(-1,2), z_obs)

  mu, std = gpr.predict(x1x2.reshape(-1, 2), return_std=True)

  Zmu = np.reshape(mu, (50, 50))
  X0p = x1x2[:, 0].reshape(50, 50)
  X1p = x1x2[:, 1].reshape(50, 50)
  plt.pcolormesh(X0p, X1p, Zmu)
  plt.title('Your Face Space')
  plt.xlabel('Smile')
  plt.ylabel('Age')
  plt.xlim([-1,1])
  plt.ylim([-1,1])
  plt.scatter(0,0,marker='X', c='red')

  xymax = np.where(Zmu==Zmu.max()) 
  xmax = xymax[0][0]
  ymax = xymax[1][0]  

  plt.scatter(X0p[xmax,0], X1p[0,ymax], marker='X', c='blue')
  plt.colorbar
  plt.show()
  print(' Marked in red is what the computer thinks is the closest face to yours.')
  print(' Marked in blue is what you rated to be the closest face to yours.')

  run = widgets.ToggleButton(
  value=False,
  description="Faces",
  disabled=False,
  button_style='info', # 'success', 'info', 'warning', 'danger' or ''
  tooltip='Click to see the highest face.',
  layout= Layout(border = '1px solid black')
  )

  display(HTML("<marquee style='width: 40%; color: green;'><b> By clicking on the next button you will see the face you rated highest.</b></marquee>"))
  display(run)

  def run_clicked(obj): # Clicked to see results
      run.close()
      show_face(latent_vector, smile_direction, age_direction,
                X0p[xmax,0], X1p[0,ymax])

  run.observe(run_clicked, 'value')

def show_face(latent_vector, direction1, direction2, x, y):
  # Show faces
  def plot_final_face(latent_vector, direction1, direction2, x, y):
    clear_output(wait=True)
    plt.figure(figsize=(10,20))
    plt.subplot(121)
    plt.title('Your "artificial" face')
    plt.imshow(generate_image(latent_vector))
    new_latent_vector = latent_vector.copy()
    new_latent_vector[:8] = (latent_vector + x*direction1)[:8]
    new_latent_vector[:8] = (new_latent_vector + y*direction2)[:8]
    plt.axis('off')
    plt.subplot(122)
    plt.title('Face rated the highest')
    plt.imshow(generate_image(new_latent_vector))
    plt.axis('off')
    plt.show()

  plot_final_face(latent_vector, smile_direction, age_direction, x*2, y*2)
  
  if y > 0:
    print('You identified yourself {} younger!'.format(y))
  if y < 0:
    print('You identified yourself {} older!'.format(y))
  if x > 0:
    print('You identified yourself {} happier!'.format(x))
  if x < 0:
    print('You identified yourself {} angrier!'.format(x))

  display(HTML("<marquee style='width: 30%; color: green;'><b> Thank you for playing! To finish, click in the button below. </b></marquee>"))
  run = widgets.ToggleButton(
  value=False,
  description="End",
  disabled=False,
  button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
  tooltip='Click here to finish the game.',
  layout= Layout(border = '1px solid black')
  )

  display(run)

  def run_clicked(obj): # Clicked to see results
      run.close()
      !kill -9 -1
  run.observe(run_clicked, 'value')

.

.

.

.

.

.

.

In [ ]:
#@title To try again, first click play on this cell. {display-mode: "form"}

!kill -9 -1

print('You can now play again the first cell.')
